In [1]:
import pandas as pd

In [2]:
# add tconst, average rating, and number of ratings
ratings_df = pd.read_csv('title.ratings.tsv', delimiter='\t')

In [3]:
# add title type, adult or not, release year, runtime, and genre
basics_df = pd.read_csv('title.basics.tsv', delimiter='\t')
basics_df = basics_df.drop(["primaryTitle", "originalTitle", "endYear"], axis=1)

/var/folders/d4/ypbwpx292519124yys2bmqwh0000gn/T/ipykernel_11535/3944222072.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics_df = pd.read_csv('title.basics.tsv', delimiter='\t')


In [4]:
# merge ratings and basics
merged1_df = pd.merge(ratings_df, basics_df, on='tconst', how='inner')


# filter number of votes
merged1_df = merged1_df[merged1_df['numVotes'] >= 20000]

# filter 2000-2023
merged1_df['startYear'] = pd.to_numeric(merged1_df['startYear'], errors='coerce')
merged1_df = merged1_df[merged1_df['startYear'] >= 2000]

# filter to movies only
merged1_df = merged1_df[merged1_df['titleType'] == 'movie']

# filter to movies only
merged1_df = merged1_df[merged1_df['isAdult'] == 0]

# make sure runtime is available
merged1_df = merged1_df[merged1_df['runtimeMinutes'] != '\\N']

print(merged1_df.shape[0])
merged1_df.head()

4836


,tconst,averageRating,numVotes,titleType,isAdult,startYear,runtimeMinutes,genres
17871,tt0035423,6.4,86345,movie,0,2001,118,"Comedy,Fantasy,Romance"
88451,tt0118589,2.4,23726,movie,0,2001,104,"Drama,Music,Romance"
88533,tt0118694,8.1,155077,movie,0,2000,98,"Drama,Romance"
89826,tt0120202,6.7,21643,movie,0,2000,105,"Comedy,Drama"
90181,tt0120630,7.1,197221,movie,0,2000,84,"Adventure,Animation,Comedy"


In [5]:
# add director, writor
crew_df = pd.read_csv('title.crew.tsv', delimiter='\t')

In [6]:
# merge crew with merge1
merged2_df = pd.merge(merged1_df, crew_df, on='tconst', how='inner')
merged2_df['writers'] = merged2_df['writers'].replace('\\N', 'doc')

print(merged2_df.shape[0])
merged2_df.head()

4836


,tconst,averageRating,numVotes,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers
0,tt0035423,6.4,86345,movie,0,2001,118,"Comedy,Fantasy,Romance",nm0003506,"nm0737216,nm0003506"
1,tt0118589,2.4,23726,movie,0,2001,104,"Drama,Music,Romance",nm0193554,"nm0921985,nm0486824"
2,tt0118694,8.1,155077,movie,0,2000,98,"Drama,Romance",nm0939182,nm0939182
3,tt0120202,6.7,21643,movie,0,2000,105,"Comedy,Drama",nm0000519,nm0000519
4,tt0120630,7.1,197221,movie,0,2000,84,"Adventure,Animation,Comedy","nm0520485,nm0661910","nm0520485,nm0661910,nm0456732,nm0123666,nm0640976"


In [7]:
# add principles
actor_df = pd.read_csv('title.principals.tsv', delimiter='\t')
actor_df = actor_df.drop(["job", "characters"], axis=1)

In [8]:
num_unique_names = actor_df['category'].unique()
print(num_unique_names)

['self' 'director' 'cinematographer' 'composer' 'producer' 'editor'
 'actor' 'actress' 'writer' 'production_designer' 'archive_footage'
 'archive_sound']


In [11]:
# merge actors with merge1
merged3_df = pd.merge(merged2_df, actor_df, on='tconst', how='inner')
merged3_df = merged3_df.rename(columns={'nconst': 'actor'})

# filter number to actors
merged3_df = merged3_df[(merged3_df['category'] != 'actor') | (merged3_df['category'] != 'actress') 
                        | (merged3_df['category'] != 'self') | (merged3_df['category'] != 'archive_footage')]

# filter to principle cast
merged3_df = merged3_df[merged3_df['ordering'] <2]

print(merged3_df.shape[0])
merged3_df.head()

4836


,tconst,averageRating,numVotes,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers,ordering,actor,category
1,tt0035423,6.4,86345,movie,0,2001,118,"Comedy,Fantasy,Romance",nm0003506,"nm0737216,nm0003506",1,nm0000212,actress
11,tt0118589,2.4,23726,movie,0,2001,104,"Drama,Music,Romance",nm0193554,"nm0921985,nm0486824",1,nm0001014,actress
21,tt0118694,8.1,155077,movie,0,2000,98,"Drama,Romance",nm0939182,nm0939182,1,nm0504897,actor
31,tt0120202,6.7,21643,movie,0,2000,105,"Comedy,Drama",nm0000519,nm0000519,1,nm0000450,actor
41,tt0120630,7.1,197221,movie,0,2000,84,"Adventure,Animation,Comedy","nm0520485,nm0661910","nm0520485,nm0661910,nm0456732,nm0123666,nm0640976",1,nm0000154,actor


In [12]:
# add language
lang_df = pd.read_csv('title.akas.tsv', delimiter='\t')
lang_df = lang_df[['titleId','language']]
lang_df = lang_df.rename(columns={'titleId': 'tconst'})

# merge language with merge3
merged4_df = pd.merge(merged3_df, lang_df, on='tconst', how='inner')

# filter en
merged4_df = merged4_df[merged4_df['language'] == 'en']
merged4_df = merged4_df.drop_duplicates(subset='tconst', keep='first')


print(merged4_df.shape[0])
merged4_df.head()

/var/folders/d4/ypbwpx292519124yys2bmqwh0000gn/T/ipykernel_11535/2796669253.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  lang_df = pd.read_csv('title.akas.tsv', delimiter='\t')


4777


,tconst,averageRating,numVotes,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers,ordering,actor,category,language
4,tt0035423,6.4,86345,movie,0,2001,118,"Comedy,Fantasy,Romance",nm0003506,"nm0737216,nm0003506",1,nm0000212,actress,en
49,tt0118589,2.4,23726,movie,0,2001,104,"Drama,Music,Romance",nm0193554,"nm0921985,nm0486824",1,nm0001014,actress,en
69,tt0118694,8.1,155077,movie,0,2000,98,"Drama,Romance",nm0939182,nm0939182,1,nm0504897,actor,en
147,tt0120202,6.7,21643,movie,0,2000,105,"Comedy,Drama",nm0000519,nm0000519,1,nm0000450,actor,en
170,tt0120630,7.1,197221,movie,0,2000,84,"Adventure,Animation,Comedy","nm0520485,nm0661910","nm0520485,nm0661910,nm0456732,nm0123666,nm0640976",1,nm0000154,actor,en


In [13]:
final_df = merged4_df.drop(["titleType", "isAdult", "startYear", "ordering","language","category", "numVotes", "writers"], axis=1)

final_df.to_csv("projectData.csv", index=False)
print(final_df.shape[0])
final_df.head()

4777


,tconst,averageRating,runtimeMinutes,genres,directors,actor
4,tt0035423,6.4,118,"Comedy,Fantasy,Romance",nm0003506,nm0000212
49,tt0118589,2.4,104,"Drama,Music,Romance",nm0193554,nm0001014
69,tt0118694,8.1,98,"Drama,Romance",nm0939182,nm0504897
147,tt0120202,6.7,105,"Comedy,Drama",nm0000519,nm0000450
170,tt0120630,7.1,84,"Adventure,Animation,Comedy","nm0520485,nm0661910",nm0000154
